GOAL: Export and downlaod data from ArcGIS Online/Portal Hosted Feature Service
TODO: determine if service is configured to allow Export

In [ ]:
import arcpy
import os
import time
from arcgis.gis import GIS
from getpass import getpass
from zipfile import ZipFile

In [ ]:
response = input(f'Do you want to use the active portal? {(arcpy.GetActivePortalURL())} [y or n]')
if response.upper() in "YES":
    #arcpy.SignInToPortal(input("Enter portal address "), username=(input("Enter User Name ")), password=(getpass()))
    gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
else:
    arcpy.SignInToPortal(input("Enter portal address "), username=(input("Enter User Name ")), password=getpass())
    #gis = GIS(input("Enter portal address "), input("Enter User Name "), getpass())
portal_desc = arcpy.GetPortalDescription()
print(f'Portal Name {portal_desc["portalName"]}\nConnected to {portal_desc["name"]} as user {portal_desc["user"]["username"]}')

In [ ]:
# search and list all items hosted on the portal
query=f'owner:{portal_desc["user"]["username"]}'
#query=''
itemType="Feature Layer"
sortField="title"
sortOrder="asc"
# default max__items is 10
maxItems=100
searchResult = gis.content.search(query,itemType,sortField,sortOrder,maxItems)
for i in range(len(searchResult)):
    display(searchResult[i])
    downloadResult = input(f'Download {(searchResult[i].title)} [Y,N]')
    if downloadResult.upper() in "YES":
        sdItem = searchResult[i]
        print(sdItem.title, sdItem.id, sdItem.modified, sdItem.access)
        break
if downloadResult.upper() not in "YES":
    print(f'All Feature Layers hosted on {portal_desc["portalName"]} were presented')

In [ ]:
data_item = gis.content.get(sdItem.id)
data_item

In [ ]:
fgdbItem = data_item.export(sdItem.title + '_fgdb', 'File Geodatabase', parameters=None, wait='True')
#TODO use this solution instead of timer 
#https://community.esri.com/thread/208467-download-feature-service-as-file-geodatabase
time.sleep(120)
gis.content.get(fgdbItem['exportItemId']).download(save_path = r'c:\users\friendde\Downloads')

In [ ]:
zipFile = 'c://users//friendde//Downloads//' + sdItem.title + '_fgdb.zip'
zipFile

In [ ]:
extractPath, ext = os.path.splitext(zipFile)
extractPath

In [ ]:
zf = ZipFile(zipFile)
zf.extractall(path=extractPath)

In [ ]:
file_list = os.listdir(extractPath)
file_list

TODO: Delete hosted exported fgdb? Delete local copies?